# 잔존율 / 이탈률 (BIGQUERY)

In [ ]:
with data_1st as (
    SELECT event_name, count(*) as event_count 
  FROM `second-hand-test.analytics_440602171.events_*`
  where event_date between '20240518' and '20240520'
  group by event_name
  order by event_name
)

SELECT
    event_name,
    event_count,
    ((LAG(event_count,1) OVER (ORDER BY event_count DESC, event_name ASC)-event_count)/LAG(event_count,1) OVER (ORDER BY event_count DESC, event_name ASC)) as bounce_rate,
    event_count/FIRST_VALUE(event_count) OVER (ORDER BY event_count DESC, event_name ASC) as retention_rate
from data_1st
where event_name LIKE '%Enter%' OR event_name LIKE 'page_view'
order by event_count desc, event_name asc

# 퍼널 전환율/이탈률 (BIGQUERY)

In [ ]:
# -> `a_impression: 노출 (page_view)`
# -> `b_lead: 유입 (시작버튼 누른 사용자)`
# -> `c_engagement: 응답 (Q8까지 도달한 사용자)`
# -> `d_reach: 공유 (결과페이지까지 본 사용자)`

In [ ]:
WITH event_counts AS (
    SELECT 
        count(distinct case when event_name = "page_view" then user_pseudo_id END) as impression,
        count(distinct case when event_name = "Enter_Q1" then user_pseudo_id END) as lead_,
        count(distinct case when event_name LIKE "q7_a%" then user_pseudo_id END) as engagement,
        count(distinct case when event_name = "Enter_ResultPage" then user_pseudo_id END) as reach
    FROM `second-hand-test.analytics_440602171.events_*`
    WHERE event_date BETWEEN '20240518' AND '20240520'
), funnel_data as (
  SELECT 'a_impression' as event_name, impression as event_count FROM event_counts
  UNION ALL
  SELECT 'b_lead', lead_ FROM event_counts
  UNION ALL
  SELECT 'c_engagement', engagement FROM event_counts
  UNION ALL
  SELECT 'd_reach', reach FROM event_counts
)

SELECT 
  event_name,
  event_count,
  ROUND((event_count / LAG(event_count) OVER (ORDER BY event_name)) * 100, 2) AS conversion_rate,
  100 - ROUND((event_count / LAG(event_count) OVER (ORDER BY event_name)) * 100, 2) AS churn_rate
FROM funnel_data
ORDER BY event_name

# 퍼널 분석 차트 (Python)

In [ ]:
json_data = [
    {"event_name": "a_impression", "event_count": 55},
    {"event_name": "b_lead", "event_count": 46},
    {"event_name": "c_engagement", "event_count": 32},
    {"event_name": "d_reach", "event_count": 27}
]

df = pd.DataFrame(json_data)

fig = px.funnel(df, y='event_name', x='event_count')
fig.show()

# AB test

In [3]:
import scipy.stats as stats

# 원본 데이터
succ_a = 43  # 대조그룹 A에서 시작 버튼을 클릭한 사람 수
tot_a = 63   # 대조그룹 A의 시작 페이지 view 수
fail_a = tot_a - succ_a  # 대조그룹 A에서 클릭하지 않은 사람 수

succ_b = 36  # 실험그룹 B에서 시작 버튼을 클릭한 사람 수
tot_b = 48   # 실험그룹 B의 시작 페이지 view 수
fail_b = tot_b - succ_b  # 실험그룹 B에서 클릭하지 않은 사람 수

# 7배로 증가시킨 데이터
succ_a_7x = succ_a * 7
tot_a_7x = tot_a * 7
fail_a_7x = tot_a_7x - succ_a_7x

succ_b_7x = succ_b * 7
tot_b_7x = tot_b * 7
fail_b_7x = tot_b_7x - succ_b_7x

# 원본 데이터로 카이제곱 검정
table_orig = [[succ_a, fail_a], [succ_b, fail_b]]
chi2_orig, p_val_orig, _, _ = stats.chi2_contingency(table_orig)

# 7배 증가된 데이터로 카이제곱 검정
table_7x = [[succ_a_7x, fail_a_7x], [succ_b_7x, fail_b_7x]]
chi2_7x, p_val_7x, _, _ = stats.chi2_contingency(table_7x)

# 결과 출력
print("원본 데이터 결과:")
print(f"카이제곱 통계량: {chi2_orig:.4f}")
print(f"p-value: {p_val_orig:.4f}\n")

print("7배 증가된 데이터 결과:")
print(f"카이제곱 통계량: {chi2_7x:.4f}")
print(f"p-value: {p_val_7x:.4f}")

원본 데이터 결과:
카이제곱 통계량: 0.3202
p-value: 0.5715

7배 증가된 데이터 결과:
카이제곱 통계량: 3.9074
p-value: 0.0481
